In [2]:
!pip install stanza
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import gc
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer

In [4]:
test_set = pd.read_csv("tsd_test.csv") # Read the test dataset
test_set['spans'] = test_set['spans'].apply(lambda x : json.loads(x)) # Extract the spans to a List
train_set = pd.read_csv("tsd_train.csv") # Read the training dataset
train_set['spans'] = train_set['spans'].apply(lambda x : json.loads(x)) # Extract the spans to a List

In [43]:
toxic_span_dataset = test_set.append(train_set, ignore_index = True)
toxic_span_dataset['text'] = toxic_span_dataset['text'].apply(lambda x : x.lower())

In [31]:
print(test_set.dtypes)

spans    object
text     object
dtype: object


In [38]:
print(test_set)

                                                  spans  \
0     [84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...   
1                              [81, 82, 83, 84, 85, 86]   
2                                                    []   
3                                                    []   
4                                                    []   
...                                                 ...   
1995  [4, 5, 6, 7, 8, 70, 71, 72, 73, 74, 75, 76, 77...   
1996                               [23, 24, 25, 26, 27]   
1997  [157, 158, 159, 160, 161, 162, 163, 164, 165, ...   
1998                                                 []   
1999                               [828, 829, 830, 831]   

                                                   text  
0     That's right. They are not normal. And I am st...  
1     "Watch people die from taking away their healt...  
2     tens years ago i contacted the PDR and suggest...  
3     The parallels between the ANC and the Sicilian...  
4

In [32]:
print(test_set['spans'])

0       [84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...
1                                [81, 82, 83, 84, 85, 86]
2                                                      []
3                                                      []
4                                                      []
                              ...                        
1995    [4, 5, 6, 7, 8, 70, 71, 72, 73, 74, 75, 76, 77...
1996                                 [23, 24, 25, 26, 27]
1997    [157, 158, 159, 160, 161, 162, 163, 164, 165, ...
1998                                                   []
1999                                 [828, 829, 830, 831]
Name: spans, Length: 2000, dtype: object


In [33]:
print(type(test_set))

<class 'pandas.core.frame.DataFrame'>


In [34]:
print(type(test_set['spans']))

<class 'pandas.core.series.Series'>


In [35]:
print(test_set['spans'])

0       [84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...
1                                [81, 82, 83, 84, 85, 86]
2                                                      []
3                                                      []
4                                                      []
                              ...                        
1995    [4, 5, 6, 7, 8, 70, 71, 72, 73, 74, 75, 76, 77...
1996                                 [23, 24, 25, 26, 27]
1997    [157, 158, 159, 160, 161, 162, 163, 164, 165, ...
1998                                                   []
1999                                 [828, 829, 830, 831]
Name: spans, Length: 2000, dtype: object


In [36]:
print(type(test_set['spans'][0]))

<class 'list'>


In [37]:
print(test_set['spans'][0])

[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146]


In [40]:
print(type(test_set['text']))

<class 'pandas.core.series.Series'>


In [39]:
print(test_set['text'])

0       That's right. They are not normal. And I am st...
1       "Watch people die from taking away their healt...
2       tens years ago i contacted the PDR and suggest...
3       The parallels between the ANC and the Sicilian...
4       Intel Community: ‘How can we work for a Presid...
                              ...                        
1995    hey loser change your name to something more a...
1996    And you are a complete moron who obviously doe...
1997    Such vitriol from the left.  Who would have th...
1998    It is now time for most of you to expand your ...
1999    Why does this author think she can demand, or ...
Name: text, Length: 2000, dtype: object


In [41]:
print(type(test_set['text'][0]))

<class 'str'>


In [42]:
print(test_set['text'][0])

That's right. They are not normal. And I am starting from the premise that they are ABNORMAL. Proceed wth the typical racist, bigot, sexist rubbish.
Thanks!


In [45]:
print(train_set)

                                                  spans  \
0     [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...   
1                          [33, 34, 35, 36, 37, 38, 39]   
2                                          [0, 1, 2, 3]   
3             [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]   
4                          [32, 33, 34, 35, 36, 37, 38]   
...                                                 ...   
7934                                     [8, 9, 10, 11]   
7935  [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5...   
7936  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
7937                            [5, 6, 7, 8, 9, 10, 11]   
7938  [106, 107, 108, 109, 110, 169, 170, 171, 172, ...   

                                                   text  
0     Another violent and aggressive immigrant killi...  
1     I am 56 years old, I am not your fucking junio...  
2                     Damn, a whole family. Sad indeed.  
3     What a knucklehead. How can anyone not know th...  
4

In [46]:
print(len(train_set))

7939


In [47]:
print(len(toxic_span_dataset))

9939


In [44]:
print(toxic_span_dataset)

                                                  spans  \
0     [84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...   
1                              [81, 82, 83, 84, 85, 86]   
2                                                    []   
3                                                    []   
4                                                    []   
...                                                 ...   
9934                                     [8, 9, 10, 11]   
9935  [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5...   
9936  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
9937                            [5, 6, 7, 8, 9, 10, 11]   
9938  [106, 107, 108, 109, 110, 169, 170, 171, 172, ...   

                                                   text  
0     that's right. they are not normal. and i am st...  
1     "watch people die from taking away their healt...  
2     tens years ago i contacted the pdr and suggest...  
3     the parallels between the anc and the sicilian...  
4

In [5]:
def createNEROutputs(texts, spans, max_length, tokenizer):
    outputs = []
    for text,span in zip(texts,spans):
        output = np.zeros(max_length*3,dtype=np.float).reshape((max_length,3))
        tokens = tokenizer.tokenize(text)[:max_length]
        length = 0
        start = True
        for i in range(len(tokens),max_length):
            output[i,0] = 1.0
        for index,token in enumerate(tokens):
            sub = True
            if "Ġ" in token:
                sub = False
                token = token[1:]
            if not start:
                next_index = text[length:].find(token)
                if next_index == 0:
                    sub = True
                length += next_index
            # if length in span and not sub:
            #     output[index,2] = 1.0
            #     output[index,0] = 0.0
            if length in span:
                output[index,2] = 1.0
                output[index,0] = 0.0
            else:
                output[index,1] = 1.0
                output[index,0] = 0.0
            length += len(token)
            start = False
        outputs.append(output)
    return np.array(outputs)

9700*400*3

In [6]:
def NERGetIndicesSingleText(outputs,text,tokenizer):
    outputs = tf.argmax(outputs,axis=-1)
    tokens = tokenizer.tokenize(text)
    index = 0
    indexes = []
    sub = False
    prev = False
    for token,output in zip(tokens,outputs):
        if token[0] == "Ġ":
            token = token[1:]
            sub = False
        elif token.isalpha():
            sub = True
        else:
            sub = False
        temp_index = text[index:].find(token)
        temp_start = index+temp_index
        if output == 2 or (sub and prev and output != 0):
            prev = True
            indexes = indexes + list(range(temp_start,temp_start+len(token)))
        else:
            prev = False
        index = temp_start+len(token)
    return np.array(indexes)

In [7]:
def createIndicesForNERModel(predicts,texts,tokenizer):
    outputs = []
    for text,pred in zip(texts,predicts):
         indices = NERGetIndicesSingleText(pred,text,tokenizer)
         outputs.append(indices)
    return outputs

In [8]:
def f1(preds,trues):
    if len(trues) == 0:
        return 1. if len(preds) == 0 else 0.
    if len(preds) == 0:
        return 0.
    predictions_set = set(preds)
    gold_set = set(trues)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)


In [9]:
def avg_f1(preds,trues):
    avg_f1_total = 0.0
    for pred,true in zip(preds,trues):
        avg_f1_total += f1(pred,true)
    return avg_f1_total/len(preds)

In [10]:
class F1Metric(callbacks.Callback):
    def __init__(self,inputs,labels,spans,texts,test=True):
        self.inputs = inputs
        self.spans = spans
        self.tokenizer = tokenizer
        self.texts = texts
        self.test = test

    def on_epoch_end(self, epoch, logs={}):
        preds = self.model.predict(self.inputs,verbose=0)
        indices = createIndicesForNERModel(preds,texts,tokenizer)
        f1 = avg_f1(indices,self.spans)
        if self.test:
            print()
            print("test f1 = "+str(f1))
        else:
            print()
            print("train f1 = "+str(f1))

In [11]:
def createInputForNER(texts,max_length,tokenizer):
    input_length = []
    for text in texts:
        input_length.append(min(max_length,len(tokenizer.tokenize(text))))
    tokens = tokenizer(texts,padding="max_length",max_length=max_length,return_tensors="tf",truncation=True)
    data = [np.array(tokens['input_ids']),np.array(tokens['attention_mask']),np.array(input_length)]
    return data

In [12]:
!pip install tensorflow-addons==0.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.0 MB/s 


In [13]:
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.layers import Layer, InputSpec
import tensorflow_addons as tfa

try:
    from tensorflow.contrib.crf import crf_decode
except ImportError:
    from tensorflow.python.framework import dtypes
    from tensorflow.python.ops import array_ops, gen_array_ops, math_ops, rnn, rnn_cell


    class CrfDecodeForwardRnnCell(rnn_cell.RNNCell):
        def __init__(self, transition_params):
            self._transition_params = array_ops.expand_dims(transition_params, 0)
            self._num_tags = transition_params.get_shape()[0]

        @property
        def state_size(self):
            return self._num_tags

        @property
        def output_size(self):
            return self._num_tags

        def __call__(self, inputs, state, scope=None):
            state = array_ops.expand_dims(state, 2)  # [B, O, 1]
            transition_scores = state + self._transition_params  # [B, O, O]
            new_state = inputs + math_ops.reduce_max(transition_scores, [1])  # [B, O]
            backpointers = math_ops.argmax(transition_scores, 1)
            backpointers = math_ops.cast(backpointers, dtype=dtypes.int32)  # [B, O]
            return backpointers, new_state


    class CrfDecodeBackwardRnnCell(rnn_cell.RNNCell):
        def __init__(self, num_tags):
            self._num_tags = num_tags

        @property
        def state_size(self):
            return 1

        @property
        def output_size(self):
            return 1

        def __call__(self, inputs, state, scope=None):
            state = array_ops.squeeze(state, axis=[1])  # [B]
            batch_size = array_ops.shape(inputs)[0]
            b_indices = math_ops.range(batch_size)  # [B]
            indices = array_ops.stack([b_indices, state], axis=1)  # [B, 2]
            new_tags = array_ops.expand_dims(
                gen_array_ops.gather_nd(inputs, indices),  # [B]
                axis=-1)  # [B, 1]

            return new_tags, new_tags


    def crf_decode(potentials, transition_params, sequence_length):
        num_tags = potentials.get_shape()[2]

        # Computes forward decoding. Get last score and backpointers.
        crf_fwd_cell = CrfDecodeForwardRnnCell(transition_params)
        initial_state = array_ops.slice(potentials, [0, 0, 0], [-1, 1, -1])
        initial_state = array_ops.squeeze(initial_state, axis=[1])  # [B, O]
        inputs = array_ops.slice(potentials, [0, 1, 0], [-1, -1, -1])  # [B, T-1, O]
        backpointers, last_score = rnn.dynamic_rnn(
            crf_fwd_cell,
            inputs=inputs,
            sequence_length=sequence_length - 1,
            initial_state=initial_state,
            time_major=False,
            dtype=dtypes.int32)  # [B, T - 1, O], [B, O]
        backpointers = gen_array_ops.reverse_sequence(backpointers, sequence_length - 1, seq_dim=1)  # [B, T-1, O]

        # Computes backward decoding. Extract tag indices from backpointers.
        crf_bwd_cell = CrfDecodeBackwardRnnCell(num_tags)
        initial_state = math_ops.cast(math_ops.argmax(last_score, axis=1), dtype=dtypes.int32)  # [B]
        initial_state = array_ops.expand_dims(initial_state, axis=-1)  # [B, 1]
        decode_tags, _ = rnn.dynamic_rnn(
            crf_bwd_cell,
            inputs=backpointers,
            sequence_length=sequence_length - 1,
            initial_state=initial_state,
            time_major=False,
            dtype=dtypes.int32)  # [B, T - 1, 1]
        decode_tags = array_ops.squeeze(decode_tags, axis=[2])  # [B, T - 1]
        decode_tags = array_ops.concat([initial_state, decode_tags], axis=1)  # [B, T]
        decode_tags = gen_array_ops.reverse_sequence(decode_tags, sequence_length, seq_dim=1)  # [B, T]

        best_score = math_ops.reduce_max(last_score, axis=1)  # [B]
        return decode_tags, best_score


class CRFLayer(Layer):

    def __init__(self, transition_params=None, **kwargs):
        super(CRFLayer, self).__init__(**kwargs)
        self.transition_params = transition_params
        self.input_spec = [InputSpec(ndim=3), InputSpec(ndim=2)]
        self.supports_masking = True

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape[0]) == 3

        return input_shape[0]

    def build(self, input_shape):
        assert len(input_shape) == 2
        assert len(input_shape[0]) == 3
        assert len(input_shape[1]) == 2
        n_steps = input_shape[0][1]
        n_classes = input_shape[0][2]
        assert n_steps is None or n_steps >= 2

        self.transition_params = self.add_weight(shape=(n_classes, n_classes),
                                                 initializer='uniform',
                                                 name='transition')
        self.input_spec = [InputSpec(dtype=K.floatx(), shape=(None, n_steps, n_classes)),
                           InputSpec(dtype='int32', shape=(None, 1))]
        self.built = True

    def viterbi_decode(self, potentials, sequence_length):
        decode_tags, best_score = crf_decode(potentials, self.transition_params, sequence_length)
        return decode_tags

    def call(self, inputs, mask=None, **kwargs):
        inputs, sequence_lengths = inputs
        self.sequence_lengths = K.flatten(sequence_lengths)
        y_pred = self.viterbi_decode(inputs, self.sequence_lengths)
        nb_classes = self.input_spec[0].shape[2]
        y_pred_one_hot = K.one_hot(y_pred, nb_classes)

        return K.in_train_phase(inputs, y_pred_one_hot)

    def loss(self, y_true, y_pred):
        y_true = K.cast(K.argmax(y_true, axis=-1), dtype='int32')
        log_likelihood, self.transition_params = tfa.text.crf.crf_log_likelihood(
            y_pred, y_true, self.sequence_lengths, self.transition_params)
        loss = tf.reduce_mean(-log_likelihood)

        return loss

    def get_config(self):
        config = {
            'transition_params': K.eval(self.transition_params),
        }
        base_config = super(CRFLayer, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))


def create_custom_objects():
    instanceHolder = {'instance': None}

    class ClassWrapper(CRFLayer):
        def __init__(self, *args, **kwargs):
            instanceHolder['instance'] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)

    def loss(*args):
        method = getattr(instanceHolder['instance'], 'loss')
        return method(*args)

    return {'CRFLayer': ClassWrapper, 'loss': loss}

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.2 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [14]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# base_model = TFBertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [15]:
def createToxicModelWithGivenBaseModel(max_input_length,base_model):
    input_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_input_ids",dtype=tf.int32)
    input_attention_mask_layer = layers.Input(shape=(max_input_length,),name="encoder_attention_mask",dtype=tf.int32)
    input_length = layers.Input(shape=(1,),name="length",dtype=tf.int32)
    base_model.trainable = True
    base_model = base_model(input_ids_layer,attention_mask=input_attention_mask_layer,return_dict=True)
    output = layers.Dropout(0.1)(base_model.last_hidden_state)
    output = layers.Dense(1024,activation="relu")(output)
    output = layers.Dropout(0.1)(output)
    output = layers.Dense(1024,activation="relu")(output)
    output = layers.Dense(3,activation="linear")(output)
    crf = CRFLayer()
    output = crf(inputs=[output,input_length])
    model = models.Model(inputs=[input_ids_layer,input_attention_mask_layer,input_length],outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=3e-5),loss=crf.loss,metrics=['accuracy'])
    return model

In [16]:
max_length = 400

In [17]:
texts = toxic_span_dataset['text'].to_numpy()
targets = createNEROutputs(texts, toxic_span_dataset['spans'], max_length, tokenizer)
all_spans = toxic_span_dataset['spans'].to_numpy()
result_test = []
result_train = []
kf = KFold(n_splits=5)
train_test_indices = []
for train_index,test_index in kf.split(texts):
    train_test_indices.append((train_index,test_index))

<ipython-input-5-5598d9551a20>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  output = np.zeros(max_length*3,dtype=np.float).reshape((max_length,3))


In [18]:
train_index,test_index = train_test_indices.pop()
x_train , x_test = list(texts[train_index]) , list(texts[test_index])
y_train , y_test = targets[train_index] , targets[test_index]
model = None
base_model = None
gc.collect()
tf.keras.backend.clear_session()
base_model = TFRobertaModel.from_pretrained('roberta-base')
model = createToxicModelWithGivenBaseModel(max_length,base_model)
train_data = createInputForNER(x_train,max_length,tokenizer)
test_data = createInputForNER(x_test,max_length,tokenizer)
spans_test = all_spans[test_index]
spans_train = all_spans[train_index]
model.fit(train_data,y_train,batch_size=16,epochs=2)
preds = model.predict(test_data)
indices = createIndicesForNERModel(preds,x_test,tokenizer)
f1_toxic = avg_f1(indices,spans_test)
print("test F1 = %f"%(f1_toxic))
result_test.append(f1_toxic)
preds = model.predict(train_data)
indices = createIndicesForNERModel(preds,x_train,tokenizer)
f1_toxic = avg_f1(indices,spans_train)
print("train F1 = %f"%(f1_toxic))
result_train.append(f1_toxic)

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Epoch 1/2


497/497 [==============================] - 849s 2s/step - loss: 9.8801 - accuracy: 0.1106
Epoch 2/2
63/63 [==============================] - 73s 1s/step
test F1 = 0.652542
249/249 [==============================] - 274s 1s/step
train F1 = 0.684947
